In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [11]:
spark = SparkSession.builder \
    .appName("CSV Reader") \
    .getOrCreate()

# CSV 파일 읽기
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///home/hadoop/jupyter/dataengineering/dataset/retail-data/by-day/*")


24/06/20 03:37:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
df.limit(1).show()

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+



In [4]:
df.count()

531694

In [13]:
df.createOrReplaceTempView("dTable")

In [11]:
from pyspark.sql.functions import lit

## 문제 1
### InvoiceNo가 536365 row 값에서 InvoiceNo, Description만 출력

In [19]:
df.filter(df.InvoiceNo == "536365").\
    select(df.InvoiceNo, df.Description).\
    show()

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536365|WHITE HANGING HEA...|
|   536365| WHITE METAL LANTERN|
|   536365|CREAM CUPID HEART...|
|   536365|KNITTED UNION FLA...|
|   536365|RED WOOLLY HOTTIE...|
|   536365|SET 7 BABUSHKA NE...|
|   536365|GLASS STAR FROSTE...|
+---------+--------------------+



In [21]:
from pyspark.sql.functions import expr

df.where(expr("InvoiceNo == 536365")).\
    select("InvoiceNo", "Description").\
    show()

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536365|WHITE HANGING HEA...|
|   536365| WHITE METAL LANTERN|
|   536365|CREAM CUPID HEART...|
|   536365|KNITTED UNION FLA...|
|   536365|RED WOOLLY HOTTIE...|
|   536365|SET 7 BABUSHKA NE...|
|   536365|GLASS STAR FROSTE...|
+---------+--------------------+



In [24]:
from pyspark.sql.functions import instr, col

filter_ = col("UnitPrice") > 600
filter_2 = instr(df.Description, "POSTAGE") >= 1

In [25]:
df.limit(1).show()

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+



In [30]:
df.where(df.StockCode.isin("DOT")).\
    where(filter_ | filter_2).\
    limit(5).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   580610|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:48:00|  2196.67|      null|United Kingdom|
|   580612|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:58:00|   2114.0|      null|United Kingdom|
|   580727|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:17:00|  1599.26|   14096.0|United Kingdom|
|   580729|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:24:00|   1172.1|      null|United Kingdom|
|   580730|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:28:00|   845.23|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [4]:
sql="""
    select * from dTable limit 3
    """

In [5]:
spark.sql(sql).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [24]:
sql = """
      select *
      from dTable
      where StockCode in ("DOT")
      AND (UnitPrice > 600 OR
      instr(Description, "PostAGE") >= 1) limit 5;
      """

In [25]:
spark.sql(sql).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   580610|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:48:00|  2196.67|      null|United Kingdom|
|   580612|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:58:00|   2114.0|      null|United Kingdom|
|   580727|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:17:00|  1599.26|   14096.0|United Kingdom|
|   580729|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:24:00|   1172.1|      null|United Kingdom|
|   580730|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:28:00|   845.23|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [28]:
from pyspark.sql.functions import col, instr
filter_1 = col("StockCode") == "DOT"
filter_2 = col("UnitPrice") > 600
filter_3 = instr(col("Description"), "POSTAGE") >= 1

In [30]:
df.withColumn("isExpensive", filter_1 & (filter_2 | filter_3)).\
    where("isExpensive").\
    select("unitPrice", "isExpensive").limit(5).show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|  2196.67|       true|
|   2114.0|       true|
|  1599.26|       true|
|   1172.1|       true|
|   845.23|       true|
+---------+-----------+



In [7]:
sql = """
        select UnitPrice, StockCode = "DOT"
        AND (UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1) as isExpensive
        from dTable
        where StockCode = "DOT"
        AND (UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1) limit 10;
      """
spark.sql(sql).show()

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|  2196.67|       true|
|   2114.0|       true|
|  1599.26|       true|
|   1172.1|       true|
|   845.23|       true|
|    863.8|       true|
|  1008.96|       true|
|  1683.75|       true|
|   938.59|       true|
|   1563.0|       true|
+---------+-----------+



In [43]:
from pyspark.sql.functions import expr, pow, col
# (현재 수량 * 단가) ^2 + 5
condition1 = pow(col("Quantity") * col("UnitPrice"), 2) + 5

In [44]:
df.select(col("CustomerId"), condition1.alias("realQuantity")   ).limit(3).show()

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   14075.0|         7387.2464|
|   14075.0|             630.0|
|   14075.0|1573.1599999999996|
+----------+------------------+



In [53]:
sql = \
    """
    select CustomerId, POWER((Quantity *UnitPrice), 2) + 5
    from dTable limit 10
    """
spark.sql(sql).show()

+----------+--------------------------------------+
|CustomerId|(POWER((Quantity * UnitPrice), 2) + 5)|
+----------+--------------------------------------+
|   14075.0|                             7387.2464|
|   14075.0|                                 630.0|
|   14075.0|                    1573.1599999999996|
|   14075.0|                                 905.0|
|   14075.0|                    239.08999999999997|
|   14075.0|                    1669.6399999999999|
|   14075.0|                               1573.16|
|   14075.0|                    1650.1136000000001|
|   18180.0|                                 294.0|
|   18180.0|                                 294.0|
+----------+--------------------------------------+



In [56]:
from pyspark.sql.functions import round, bround, ceil, lit

In [77]:
# 예제 데이터 생성
data = [
    ("Alice", 23.5),
    ("Bob", 34.234),
    ("Cathy", 45.789)
]

# 스키마 정의
columns = ["name", "value"]

# DataFrame 생성
df_ex = spark.createDataFrame(data, schema=columns)


# round와 lit 사용
df_ex = df_ex.withColumn("rounded_value", round("value", 1)) \
       .withColumn("constant_value", lit(round("value", 1))) \
       .withColumn("bround_value", bround("value", 1)) \
       .withColumn("ceil_value", ceil("value"))

# 결과 출력
df_ex.show()

+-----+------+-------------+--------------+------------+----------+
| name| value|rounded_value|constant_value|bround_value|ceil_value|
+-----+------+-------------+--------------+------------+----------+
|Alice|  23.5|         23.5|          23.5|        23.5|        24|
|  Bob|34.234|         34.2|          34.2|        34.2|        35|
|Cathy|45.789|         45.8|          45.8|        45.8|        46|
+-----+------+-------------+--------------+------------+----------+



In [80]:
df.select('*').describe().limit(1).show()

24/06/20 02:40:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+---------+-----------+--------+---------+----------+-------+
|summary|InvoiceNo|StockCode|Description|Quantity|UnitPrice|CustomerID|Country|
+-------+---------+---------+-----------+--------+---------+----------+-------+
|  count|   531694|   531694|     530257|  531694|   531694|    401575| 531694|
+-------+---------+---------+-----------+--------+---------+----------+-------+



In [81]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

In [86]:
df.select(mean("UnitPrice"), count("UnitPrice"),stddev_pop("UnitPrice"), min("UnitPrice"), max("UnitPrice")).limit(1).show()

+-----------------+----------------+---------------------+--------------+--------------+
|   avg(UnitPrice)|count(UnitPrice)|stddev_pop(UnitPrice)|min(UnitPrice)|max(UnitPrice)|
+-----------------+----------------+---------------------+--------------+--------------+
|4.616183771117624|          531694|    97.64335099564296|     -11062.06|       38970.0|
+-----------------+----------------+---------------------+--------------+--------------+



In [91]:
min_price = df.agg(min("UnitPrice")).first()[0]
df.filter(col("UnitPrice") == min_price).limit(5).show()

+---------+---------+---------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|    Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+---------------+--------+-------------------+---------+----------+--------------+
|  A563186|        B|Adjust bad debt|       1|2011-08-12 14:51:00|-11062.06|      null|United Kingdom|
|  A563187|        B|Adjust bad debt|       1|2011-08-12 14:52:00|-11062.06|      null|United Kingdom|
+---------+---------+---------------+--------+-------------------+---------+----------+--------------+



In [92]:
df.stat.approxQuantile("UnitPrice", [0.5], 0.05)

[2.08]

### "UnitPrice": UnitPrice 컬럼에서 백분위수를 계산합니다.
#### [0.5]: 계산할 백분위수를 나타내는 리스트입니다. 여기서는 0.5 백분위수, 즉 중앙값(median)을 계산합니다.
#### 0.05: 상대적인 오차입니다. 이 값은 결과값이 실제 백분위수의 ±5% 내에 있음을 보장합니다. 이 값이 작을수록 계산은 더 정확하지만, 더 많은 계산 비용이 듭니다.
#### 즉, 이 코드의 목적은 UnitPrice 컬럼의 중앙값을 근사적으로 계산하는 것입니다. 중앙값은 데이터셋에서 중간에 위치하는 값으로, 데이터를 반으로 나누는 값입니다.

# 문자열

In [98]:
from pyspark.sql.functions import initcap, col, lower, upper

In [94]:
help(initcap)

Help on function initcap in module pyspark.sql.functions:

initcap(col: 'ColumnOrName') -> pyspark.sql.column.Column
    Translate the first letter of each word to upper case in the sentence.

    .. versionadded:: 1.5.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    col : :class:`~pyspark.sql.Column` or str
        target column to work on.

    Returns
    -------
    :class:`~pyspark.sql.Column`
        string with all first letters are uppercase in each word.

    Examples
    --------
    >>> spark.createDataFrame([('ab cd',)], ['a']).select(initcap("a").alias('v')).collect()
    [Row(v='Ab Cd')]



In [128]:
df.select("Description", upper("Description"), initcap("Description"), lower("Description")).limit(5).show()

+--------------------+--------------------+--------------------+--------------------+
|         Description|  upper(Description)|initcap(Description)|  lower(Description)|
+--------------------+--------------------+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|  RABBIT NIGHT LIGHT|  Rabbit Night Light|  rabbit night light|
| DOUGHNUT LIP GLOSS | DOUGHNUT LIP GLOSS | Doughnut Lip Gloss | doughnut lip gloss |
|12 MESSAGE CARDS ...|12 MESSAGE CARDS ...|12 Message Cards ...|12 message cards ...|
|BLUE HARMONICA IN...|BLUE HARMONICA IN...|Blue Harmonica In...|blue harmonica in...|
|   GUMBALL COAT RACK|   GUMBALL COAT RACK|   Gumball Coat Rack|   gumball coat rack|
+--------------------+--------------------+--------------------+--------------------+



In [103]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, 

In [115]:
df.select(ltrim(lit("  Hi    ")), 
          rtrim(lit("     Hi     ")), 
          trim(lit("     Hi    ")),
          rpad(lit("Hi"), 10, ""),
          lpad(lit("   Hi"), 6, "")).limit(3).show()

+---------------+-------------------+-----------------+--------------+----------------+
|ltrim(  Hi    )|rtrim(     Hi     )|trim(     Hi    )|rpad(Hi, 10, )|lpad(   Hi, 6, )|
+---------------+-------------------+-----------------+--------------+----------------+
|         Hi    |                 Hi|               Hi|            Hi|              Hi|
|         Hi    |                 Hi|               Hi|            Hi|              Hi|
|         Hi    |                 Hi|               Hi|            Hi|              Hi|
+---------------+-------------------+-----------------+--------------+----------------+



In [19]:
from pyspark.sql.functions import regexp_replace, regexp_extract, col

In [21]:
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"

In [ ]:
df.select( regexp_replace(col("Description"), regex_string, "COLOR").alias("color"),
            col("Description")).limit(10).show(truncate=False)

In [14]:
spark.sql(""" select regexp_replace(Description, "BLACK|WHITE|RED|GREEN|BLUE", "color")
    as color, Description from dTable
""").show(truncate=False)


+-----------------------------------+-----------------------------------+
|color                              |Description                        |
+-----------------------------------+-----------------------------------+
|RABBIT NIGHT LIGHT                 |RABBIT NIGHT LIGHT                 |
|DOUGHNUT LIP GLOSS                 |DOUGHNUT LIP GLOSS                 |
|12 MESSAGE CARDS WITH ENVELOPES    |12 MESSAGE CARDS WITH ENVELOPES    |
|color HARMONICA IN BOX             |BLUE HARMONICA IN BOX              |
|GUMBALL COAT RACK                  |GUMBALL COAT RACK                  |
|SKULLS  WATER TRANSFER TATTOOS     |SKULLS  WATER TRANSFER TATTOOS     |
|FELTCRAFT GIRL AMELIE KIT          |FELTCRAFT GIRL AMELIE KIT          |
|CAMOUFLAGE LED TORCH               |CAMOUFLAGE LED TORCH               |
|color SKULL HOT WATER BOTTLE       |WHITE SKULL HOT WATER BOTTLE       |
|ENGLISH ROSE HOT WATER BOTTLE      |ENGLISH ROSE HOT WATER BOTTLE      |
|HOT WATER BOTTLE KEEP CALM         |H

In [22]:
df.select( regexp_extract(col("Description"), regex_string, 0).alias("color"),
            col("Description")).limit(10).show(truncate=False)

+-----+-------------------------------+
|color|Description                    |
+-----+-------------------------------+
|     |RABBIT NIGHT LIGHT             |
|     |DOUGHNUT LIP GLOSS             |
|     |12 MESSAGE CARDS WITH ENVELOPES|
|BLUE |BLUE HARMONICA IN BOX          |
|     |GUMBALL COAT RACK              |
|     |SKULLS  WATER TRANSFER TATTOOS |
|     |FELTCRAFT GIRL AMELIE KIT      |
|     |CAMOUFLAGE LED TORCH           |
|WHITE|WHITE SKULL HOT WATER BOTTLE   |
|     |ENGLISH ROSE HOT WATER BOTTLE  |
+-----+-------------------------------+



## mysql 에서 데이터 가져오기

In [29]:
encore = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://43.202.5.70:3306/encore_web") \
    .option("dbtable","encore_question") \
    .option("numPartitions",5) \
    .option("user", "class5") \
    .option("password", 'EnCoRo!23') \
    .load()

In [46]:
from pyspark.sql.functions import instr
encore.where(instr("content", "점심")>=1).show()

+---+-------+-------+--------------------+---------+
| id|subject|content|         create_date|quthor_id|
+---+-------+-------+--------------------+---------+
|140|  점_심|   점심|2024-06-20 02:13:...|       50|
+---+-------+-------+--------------------+---------+



In [35]:
from pyspark.sql.functions import expr, pow, col

In [37]:
# (현재 수량 * 단가) ^2 + 5
pow(col("Quantity") * col("UnitPrice"), 2) + 5

Column<'(POWER((Quantity * UnitPrice), 2) + 5)'>